In [ ]:
import pandas as pd

# Ordenar por fecha
df = pd.read_parquet("API/historico_ventas_1_año.parquet")

df_p = pd.read_parquet("API/costo_productos.parquet")



In [ ]:
import re

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID"
}

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip().upper()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip().upper()
        else:
            # 3) Buscar prefijo de código (ej: FCAL, FMDE, etc.)
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # 🔹 Limpiar T1, T2, T3
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    return sucursal

# Aplicar
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [ ]:
df = pd.merge(df,df_p,on='product_name',how="left")
df.info()

In [ ]:
import pandas as pd
from datetime import date

# Semana y año actual
hoy = date.today()
semana_actual = 34  # lo defines manual si quieres
año_actual = hoy.year



# ================================
# 2) Agrupar ventas semanales
# ================================
df["Año"] = df["date_invoice"].dt.year
df["Semana"] = df["date_invoice"].dt.isocalendar().week

df_semana = (
    df.groupby(["Sucursal", "product_name", "Año", "Semana"])
      .agg({
          "quantity": "sum",
          "price_subtotal": "sum",
          "producto_costo_unitario": "mean"
      })
      .reset_index()
      .sort_values(["Sucursal", "product_name", "Año", "Semana"]) 
)

# ================================
# 3) Calcular ponderación personalizada
# ================================
# Asignar pesos: semanas 1-8 (80%), semanas 9-12 (20%)
df_semana = df_semana.sort_values(["Sucursal", "product_name", "Semana"])

def ema_custom(x):
    n = len(x)
    if n < 12:
        return None  # Si no hay 12 semanas completas
    ultimas = x.iloc[-4:]   # semanas 9-12
    anteriores = x.iloc[:-4]  # semanas 1-8
    return (ultimas.mean() * 0.2) + (anteriores.mean() * 0.8)

df_resultado = (
    df_semana.groupby(["Sucursal", "product_name"], as_index=False)
             .agg(EMA=("quantity", ema_custom),
                  Costo_unitario=("producto_costo_unitario", "mean"))
)


pd.set_option("display.max_colwidth", None) #muestra todo el contenido

# ================================
# 4) Agregar semana y año actuales
# ================================
df_resultado["Semana"] = semana_actual
df_resultado["Año"] = año_actual

# ================================
# 5) Calcular "Venta costo EMA"
# ================================
df_resultado["Venta_EMA"] = df_resultado["EMA"] * df_resultado["Costo_unitario"]

# ================================
# 6) Dejar columnas estilo Odoo
# ================================
df_resultado = df_resultado[[
    "Sucursal", "product_name", "Año", "Semana", "EMA",
    "Costo_unitario", "Venta_EMA"

]]


In [ ]:
df_resultado.sample(5)

In [ ]:
df_resultado = df_resultado.sort_values(by=("EMA"),ascending=False)

sede ="VALLADOLID"
df_sede = df_resultado[(df_resultado['Sucursal']==sede) & (df_resultado["product_name"].str.contains("GRASA"))]
df_sede